IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

d:\GitHub\ProgettoManutenzionePreventiva\M2\NO_FAULT2


In [3]:
guasto = 0

VOLO M2 NO FAULT

In [4]:
rcou_m2_nofault = pd.read_csv("RCOU.csv")
rcou_m2_nofault = rcou_m2_nofault.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)

potenza = 1350

rcou_m2_nofault = rcou_m2_nofault[((rcou_m2_nofault['C9'] >= potenza) & (rcou_m2_nofault['C10'] >= potenza) & (rcou_m2_nofault['C11'] >= potenza) & (rcou_m2_nofault['C12'] >= potenza) & (rcou_m2_nofault['C13'] >= potenza) & (rcou_m2_nofault['C14']>= potenza))]

rcou_m2_nofault = rcou_m2_nofault.reset_index(drop=True)

display(rcou_m2_nofault)

,TimeUS,C9,C10,C11,C12,C13,C14
0,163535671,1390,1407,1412,1385,1400,1398
1,163546021,1382,1426,1447,1359,1421,1387
2,163556077,1378,1442,1450,1368,1414,1407
3,163566849,1428,1405,1402,1431,1408,1425
4,163577867,1452,1391,1393,1450,1419,1426
...,...,...,...,...,...,...,...
7059,244588431,1556,1418,1490,1492,1444,1535
7060,244598703,1559,1408,1492,1484,1453,1521
7061,244609809,1577,1376,1489,1481,1476,1493
7062,244620798,1575,1371,1518,1443,1508,1454


In [5]:
min=rcou_m2_nofault['TimeUS'][0]
max=rcou_m2_nofault['TimeUS'][len(rcou_m2_nofault)-1]
print(max)
print(min)

244631223
163535671


In [6]:
xkf1_m2_nofault = pd.read_csv("XKF1_0.csv")
xkf1_m2_nofault = xkf1_m2_nofault.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m2_nofault = xkf1_m2_nofault[((xkf1_m2_nofault['TimeUS'] >= min) & (xkf1_m2_nofault['TimeUS'] <= max))]
xkf1_m2_nofault = xkf1_m2_nofault.reset_index(drop=True)
print(xkf1_m2_nofault)

         TimeUS  Roll  Pitch     Yaw
0     163543665  1.17  -0.58   13.28
1     163554261  1.14  -0.58   13.28
2     163564285  1.19  -0.56   13.29
3     163575443  1.23  -0.54   13.30
4     163586483  1.30  -0.52   13.28
...         ...   ...    ...     ...
7385  244586271 -4.88  -0.88  358.90
7386  244596618 -4.80  -0.96  358.86
7387  244607605 -4.77  -1.02  358.82
7388  244618752 -4.71  -1.02  358.80
7389  244629015 -4.67  -1.04  358.77

[7390 rows x 4 columns]


In [7]:
att_m2_nofault = pd.read_csv("ATT.csv")
att_m2_nofault = att_m2_nofault.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
att_m2_nofault = att_m2_nofault[((att_m2_nofault['TimeUS'] >= min) & (att_m2_nofault['TimeUS'] <= max))]
att_m2_nofault=att_m2_nofault.reset_index(drop=True)
print(att_m2_nofault)

         TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw     Yaw
0     163545292    -0.13  1.17     -0.07  -0.58   13.14   13.28
1     163555319    -0.15  1.14     -0.06  -0.58   13.14   13.28
2     163566291    -0.16  1.19     -0.05  -0.56   13.14   13.29
3     163577191    -0.18  1.23     -0.05  -0.54   13.14   13.30
4     163588078    -0.19  1.30     -0.04  -0.52   13.14   13.28
...         ...      ...   ...       ...    ...     ...     ...
7386  244587670    -2.61 -4.88     -0.90  -0.88    0.26  358.91
7387  244597835    -2.50 -4.80     -0.89  -0.96    0.26  358.87
7388  244609128    -2.42 -4.77     -0.87  -1.02    0.26  358.83
7389  244620166    -2.34 -4.71     -0.86  -1.02    0.26  358.81
7390  244630400    -2.23 -4.67     -0.85  -1.04    0.26  358.78

[7391 rows x 7 columns]


In [8]:
if len(att_m2_nofault)>len(xkf1_m2_nofault):
    to_add=att_m2_nofault[len(xkf1_m2_nofault):]
    att_m2_nofault=att_m2_nofault[:len(xkf1_m2_nofault)]

if len(xkf1_m2_nofault)>len(att_m2_nofault):
    to_add=xkf1_m2_nofault[len(att_m2_nofault):]

for idx,i in enumerate(att_m2_nofault['Roll']):
    if(xkf1_m2_nofault['Roll'][idx] != i):
        att_m2_nofault['Roll'][idx] = ((att_m2_nofault['Roll'][idx] + xkf1_m2_nofault['Roll'][idx])/2)

for idx,i in enumerate(att_m2_nofault['Pitch']):
    if(xkf1_m2_nofault['Pitch'][idx] != i):
        att_m2_nofault['Pitch'][idx] = ((att_m2_nofault['Pitch'][idx] + xkf1_m2_nofault['Pitch'][idx])/2)

for idx,i in enumerate(att_m2_nofault['Yaw']):
    if(xkf1_m2_nofault['Yaw'][idx] != i):
        att_m2_nofault['Yaw'][idx] = ((att_m2_nofault['Yaw'][idx] + xkf1_m2_nofault['Yaw'][idx])/2)

att_m2_nofault=pd.concat([att_m2_nofault,to_add])

print(att_m2_nofault)

C:\Users\mgale\AppData\Local\Temp\ipykernel_2120\600786250.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  att_m2_nofault['Roll'][idx] = ((att_m2_nofault['Roll'][idx] + xkf1_m2_nofault['Roll'][idx])/2)
C:\Users\mgale\AppData\Local\Temp\ipykernel_2120\600786250.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  att_m2_nofault['Pitch'][idx] = ((att_m2_nofault['Pitch'][idx] + xkf1_m2_nofault['Pitch'][idx])/2)
C:\Users\mgale\AppData\Local\Temp\ipykernel_2120\600786250.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://panda

         TimeUS  DesRoll   Roll  DesPitch  Pitch  DesYaw      Yaw
0     163545292    -0.13  1.170     -0.07  -0.58   13.14   13.280
1     163555319    -0.15  1.140     -0.06  -0.58   13.14   13.280
2     163566291    -0.16  1.190     -0.05  -0.56   13.14   13.290
3     163577191    -0.18  1.230     -0.05  -0.54   13.14   13.300
4     163588078    -0.19  1.300     -0.04  -0.52   13.14   13.280
...         ...      ...    ...       ...    ...     ...      ...
7386  244587670    -2.61 -4.840     -0.90  -0.92    0.26  358.885
7387  244597835    -2.50 -4.785     -0.89  -0.99    0.26  358.845
7388  244609128    -2.42 -4.740     -0.87  -1.02    0.26  358.815
7389  244620166    -2.34 -4.690     -0.86  -1.03    0.26  358.790
7390  244630400    -2.23 -4.670     -0.85  -1.04    0.26  358.780

[7391 rows x 7 columns]


In [9]:
#elettricita' che scorre nel motore considerato, al momento ne consideriamo solo 1
esc_0_m2_nofault = pd.read_csv("ESC_0.csv")
esc_0_m2_nofault = esc_0_m2_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_0_m2_nofault = esc_0_m2_nofault[((esc_0_m2_nofault['TimeUS'] >= min) & (esc_0_m2_nofault['TimeUS'] <= max))]
esc_0_m2_nofault=esc_0_m2_nofault.rename(columns={"RPM": "RPM(ESC_0)", "Curr": "Curr(ESC_0)"})

esc_1_m2_nofault = pd.read_csv("ESC_1.csv")
esc_1_m2_nofault = esc_1_m2_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_1_m2_nofault = esc_1_m2_nofault[((esc_1_m2_nofault['TimeUS'] >= min) & (esc_1_m2_nofault['TimeUS'] <= max))]
esc_1_m2_nofault=esc_1_m2_nofault.rename(columns={"RPM": "RPM(ESC_1)", "Curr": "Curr(ESC_1)"})

esc_2_m2_nofault = pd.read_csv("ESC_2.csv")
esc_2_m2_nofault = esc_2_m2_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_2_m2_nofault = esc_2_m2_nofault[((esc_2_m2_nofault['TimeUS'] >= min) & (esc_2_m2_nofault['TimeUS'] <= max))]
esc_2_m2_nofault=esc_2_m2_nofault.rename(columns={"RPM": "RPM(ESC_2)", "Curr": "Curr(ESC_2)"})

esc_3_m2_nofault = pd.read_csv("ESC_3.csv")
esc_3_m2_nofault = esc_3_m2_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_3_m2_nofault = esc_3_m2_nofault[((esc_3_m2_nofault['TimeUS'] >= min) & (esc_3_m2_nofault['TimeUS'] <= max))]
esc_3_m2_nofault=esc_3_m2_nofault.rename(columns={"RPM": "RPM(ESC_3)", "Curr": "Curr(ESC_3)"})

esc_4_m2_nofault = pd.read_csv("ESC_4.csv")
esc_4_m2_nofault = esc_4_m2_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_4_m2_nofault = esc_4_m2_nofault[((esc_4_m2_nofault['TimeUS'] >= min) & (esc_4_m2_nofault['TimeUS'] <= max))]
esc_4_m2_nofault=esc_4_m2_nofault.rename(columns={"RPM": "RPM(ESC_4)", "Curr": "Curr(ESC_4)"})

esc_5_m2_nofault = pd.read_csv("ESC_5.csv")
esc_5_m2_nofault = esc_5_m2_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_5_m2_nofault = esc_5_m2_nofault[((esc_5_m2_nofault['TimeUS'] >= min) & (esc_5_m2_nofault['TimeUS'] <= max))]
esc_5_m2_nofault=esc_5_m2_nofault.rename(columns={"RPM": "RPM(ESC_5)", "Curr": "Curr(ESC_5)"})

In [10]:
#valori di accelerazione e del giroscopio
imu_0_m2_nofault = pd.read_csv("IMU_0.csv")
imu_0_m2_nofault = imu_0_m2_nofault.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_0_m2_nofault = imu_0_m2_nofault[((imu_0_m2_nofault['TimeUS'] >= min) & (imu_0_m2_nofault['TimeUS'] <= max))]

imu_1_m2_nofault = pd.read_csv("IMU_1.csv")
imu_1_m2_nofault = imu_1_m2_nofault.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_1_m2_nofault = imu_1_m2_nofault[((imu_1_m2_nofault['TimeUS'] >= min) & (imu_1_m2_nofault['TimeUS'] <= max))]

imu_2_m2_nofault = pd.read_csv("IMU_2.csv")
imu_2_m2_nofault = imu_2_m2_nofault.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_2_m2_nofault = imu_2_m2_nofault[((imu_2_m2_nofault['TimeUS'] >= min) & (imu_2_m2_nofault['TimeUS'] <= max))]

imu_m2_nofault = pd.concat((imu_0_m2_nofault, imu_1_m2_nofault, imu_2_m2_nofault))
imu_m2_nofault=imu_m2_nofault.groupby(imu_m2_nofault.TimeUS, as_index=False).mean()

display(imu_m2_nofault)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ
0,163537245,0.073797,0.052690,0.029002,-0.099784,0.158494,-9.828200
1,163539870,0.125925,0.025622,0.027003,-0.082384,0.063078,-9.522074
2,163542660,0.030989,-0.038425,0.015617,-0.124611,-0.010491,-9.361492
3,163545177,-0.055869,-0.065410,0.012424,-0.162603,0.150488,-9.439348
4,163547584,0.044315,-0.006546,0.023296,-0.371309,0.336459,-9.679198
...,...,...,...,...,...,...,...
28162,244619831,0.128640,-0.060799,-0.040252,-0.261952,0.785947,-9.640874
28163,244622275,0.190746,-0.016192,-0.038782,-0.462853,0.624572,-9.764187
28164,244624838,0.057046,0.056984,-0.032290,-0.438465,0.580401,-9.704224
28165,244627437,0.037260,0.028643,-0.041810,-0.307790,0.681670,-9.576567


In [11]:
#sincronizzazione dei tempi tra att e imu_2
m2_nofault = pd.merge_ordered(imu_m2_nofault,att_m2_nofault)
m2_nofault=m2_nofault.fillna(method="ffill").fillna(method="bfill")
#sincronizzazione dei tempi tra la tabella ottenuta precedentemente e esc_0 (corrente che passa nel motore)
m2_nofault = pd.merge_ordered(m2_nofault, esc_0_m2_nofault)
m2_nofault = pd.merge_ordered(m2_nofault, esc_1_m2_nofault)
m2_nofault = pd.merge_ordered(m2_nofault, esc_2_m2_nofault)
m2_nofault = pd.merge_ordered(m2_nofault, esc_3_m2_nofault)
m2_nofault = pd.merge_ordered(m2_nofault, esc_4_m2_nofault)
m2_nofault = pd.merge_ordered(m2_nofault, esc_5_m2_nofault)
m2_nofault=m2_nofault.fillna(method="ffill").fillna(method="bfill")
m2_nofault = pd.merge_ordered(m2_nofault, rcou_m2_nofault)
m2_nofault=m2_nofault.fillna(method="ffill").fillna(method="bfill")

In [12]:
m2_nofault["TimeUS"] = m2_nofault["TimeUS"] - m2_nofault.iloc[0]["TimeUS"]
m2_nofault["TimeUS"] = m2_nofault["TimeUS"].astype(int)
m2_nofault["Guasto"] = guasto

In [13]:
last_cell = m2_nofault.iloc[-1, m2_nofault.columns.get_loc('TimeUS')]
print(last_cell)

81095552


In [14]:
to_be=[]
for i in range(0, last_cell,2857):
    to_be.append(i)

In [15]:
df = pd.DataFrame(to_be, columns = ['TimeUS'])

print(df)

         TimeUS
0             0
1          2857
2          5714
3          8571
4         11428
...         ...
28380  81081660
28381  81084517
28382  81087374
28383  81090231
28384  81093088

[28385 rows x 1 columns]


In [16]:
m2_nofault_final = pd.merge_ordered(m2_nofault,df)
m2_nofault_final=m2_nofault_final.fillna(method="ffill").fillna(method="bfill")
print(m2_nofault_final)

         TimeUS      GyrX      GyrY      GyrZ      AccX      AccY      AccZ  \
0             0  0.073797  0.052690  0.029002 -0.099784  0.158494 -9.828200   
1          1574  0.073797  0.052690  0.029002 -0.099784  0.158494 -9.828200   
2          1742  0.073797  0.052690  0.029002 -0.099784  0.158494 -9.828200   
3          2857  0.073797  0.052690  0.029002 -0.099784  0.158494 -9.828200   
4          4199  0.125925  0.025622  0.027003 -0.082384  0.063078 -9.522074   
...         ...       ...       ...       ...       ...       ...       ...   
75100  81091766  0.037260  0.028643 -0.041810 -0.307790  0.681670 -9.576567   
75101  81093088  0.037260  0.028643 -0.041810 -0.307790  0.681670 -9.576567   
75102  81094562  0.107283 -0.079716 -0.056382 -0.343552  0.632744 -9.387979   
75103  81094729  0.107283 -0.079716 -0.056382 -0.343552  0.632744 -9.387979   
75104  81095552  0.107283 -0.079716 -0.056382 -0.343552  0.632744 -9.387979   

       DesRoll  Roll  DesPitch  ...  Curr(ESC_4)  R

In [17]:
m2_nofault_final = m2_nofault_final[m2_nofault_final.TimeUS.isin(to_be)]

In [18]:
m2_nofault_final

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,...,Curr(ESC_4),RPM(ESC_5),Curr(ESC_5),C9,C10,C11,C12,C13,C14,Guasto
0,0,0.073797,0.052690,0.029002,-0.099784,0.158494,-9.828200,-0.13,1.17,-0.07,...,1.90,3454.0,1.83,1390.0,1407.0,1412.0,1385.0,1400.0,1398.0,0.0
3,2857,0.073797,0.052690,0.029002,-0.099784,0.158494,-9.828200,-0.13,1.17,-0.07,...,1.90,3454.0,1.83,1390.0,1407.0,1412.0,1385.0,1400.0,1398.0,0.0
5,5714,0.125925,0.025622,0.027003,-0.082384,0.063078,-9.522074,-0.13,1.17,-0.07,...,1.90,3454.0,1.83,1390.0,1407.0,1412.0,1385.0,1400.0,1398.0,0.0
7,8571,0.030989,-0.038425,0.015617,-0.124611,-0.010491,-9.361492,-0.13,1.17,-0.07,...,1.90,3454.0,1.83,1390.0,1407.0,1412.0,1385.0,1400.0,1398.0,0.0
11,11428,-0.055869,-0.065410,0.012424,-0.162603,0.150488,-9.439348,-0.13,1.17,-0.07,...,1.90,3454.0,1.83,1382.0,1426.0,1447.0,1359.0,1421.0,1387.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75089,81081660,-0.006303,0.000132,-0.044976,-0.432384,0.569364,-9.512742,-2.42,-4.74,-0.87,...,2.05,5479.0,2.69,1577.0,1376.0,1489.0,1481.0,1476.0,1493.0,0.0
75093,81084517,0.128640,-0.060799,-0.040252,-0.261952,0.785947,-9.640874,-2.34,-4.69,-0.86,...,2.05,5479.0,2.69,1577.0,1376.0,1489.0,1481.0,1476.0,1493.0,0.0
75096,81087374,0.190746,-0.016192,-0.038782,-0.462853,0.624572,-9.764187,-2.34,-4.69,-0.86,...,2.05,5479.0,2.69,1575.0,1371.0,1518.0,1443.0,1508.0,1454.0,0.0
75099,81090231,0.057046,0.056984,-0.032290,-0.438465,0.580401,-9.704224,-2.34,-4.69,-0.86,...,2.05,5424.0,2.51,1575.0,1371.0,1518.0,1443.0,1508.0,1454.0,0.0


In [19]:
from datetime import timedelta
m2_nofault_final=m2_nofault_final.reset_index(drop=True)
m2_nofault_final['TimeUS'] = pd.to_datetime(m2_nofault_final['TimeUS'], unit='us').dt.strftime('%H:%M:%S.%f')
display(m2_nofault_final)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,...,Curr(ESC_4),RPM(ESC_5),Curr(ESC_5),C9,C10,C11,C12,C13,C14,Guasto
0,00:00:00.000000,0.073797,0.052690,0.029002,-0.099784,0.158494,-9.828200,-0.13,1.17,-0.07,...,1.90,3454.0,1.83,1390.0,1407.0,1412.0,1385.0,1400.0,1398.0,0.0
1,00:00:00.002857,0.073797,0.052690,0.029002,-0.099784,0.158494,-9.828200,-0.13,1.17,-0.07,...,1.90,3454.0,1.83,1390.0,1407.0,1412.0,1385.0,1400.0,1398.0,0.0
2,00:00:00.005714,0.125925,0.025622,0.027003,-0.082384,0.063078,-9.522074,-0.13,1.17,-0.07,...,1.90,3454.0,1.83,1390.0,1407.0,1412.0,1385.0,1400.0,1398.0,0.0
3,00:00:00.008571,0.030989,-0.038425,0.015617,-0.124611,-0.010491,-9.361492,-0.13,1.17,-0.07,...,1.90,3454.0,1.83,1390.0,1407.0,1412.0,1385.0,1400.0,1398.0,0.0
4,00:00:00.011428,-0.055869,-0.065410,0.012424,-0.162603,0.150488,-9.439348,-0.13,1.17,-0.07,...,1.90,3454.0,1.83,1382.0,1426.0,1447.0,1359.0,1421.0,1387.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28380,00:01:21.081660,-0.006303,0.000132,-0.044976,-0.432384,0.569364,-9.512742,-2.42,-4.74,-0.87,...,2.05,5479.0,2.69,1577.0,1376.0,1489.0,1481.0,1476.0,1493.0,0.0
28381,00:01:21.084517,0.128640,-0.060799,-0.040252,-0.261952,0.785947,-9.640874,-2.34,-4.69,-0.86,...,2.05,5479.0,2.69,1577.0,1376.0,1489.0,1481.0,1476.0,1493.0,0.0
28382,00:01:21.087374,0.190746,-0.016192,-0.038782,-0.462853,0.624572,-9.764187,-2.34,-4.69,-0.86,...,2.05,5479.0,2.69,1575.0,1371.0,1518.0,1443.0,1508.0,1454.0,0.0
28383,00:01:21.090231,0.057046,0.056984,-0.032290,-0.438465,0.580401,-9.704224,-2.34,-4.69,-0.86,...,2.05,5424.0,2.51,1575.0,1371.0,1518.0,1443.0,1508.0,1454.0,0.0


## Analisi nel tempo e in frequenza

In [20]:
import math
#Funzione che calcola il Root Mean Square (RMS)
def rmsValue(arr, n):
    square = 0
    mean = 0.0
    root = 0.0
    #Calcola il quadrato
    for i in range(0,n):
        square += (arr[i]**2)
    #Calcola la media
    mean = (square / (float)(n))
    #Calcola la radice
    root = math.sqrt(mean)
    return root

## Funzione generica per ottenere la tabellina con le feauture nel tempo e in frequenza

In [21]:
def time_freq_feat(V):
    N=350
    height_threshold=0.0
    tab= pd.DataFrame([], columns=['tempo',f'<>({V})',f'var({V})',f'kurt({V})',f'RMS({V})',f'freq1({V})',f'freq2({V})',f'amp1({V})',f'amp2({V})'])
    for i in range(int(len(m2_nofault_final)/350)):
        V1=m2_nofault_final[str(V)].iloc[:(i+1*350)+1]
        mean=V1.mean()
        var=V1.var()
        kurt=V1.kurtosis()
        rms=rmsValue(V1,len(V1))
        yf=fft(V1.values)
        xf=fftfreq(N,1/350)
        peaks_index, properties = find_peaks(yf,height=-200,prominence=0.1)
        
        #for k in range(len(peaks_index)):
        #[print("%4.4f    \t %3.4f" %(freq[peaks_index[i]], properties['peak_heights'][i])) for i in range(len(peaks_index))]
        h_max_peak_1=np.unique(properties['peak_heights'])[-1]
        peak1_dimension = np.where(properties['peak_heights'] == h_max_peak_1)
        if (len(peak1_dimension[0])>1):
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][1]
        else:
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][0]

        h_max_peak_2=np.unique(properties['peak_heights'])[-2] 
        peak2_dimension = np.where(properties['peak_heights'] == h_max_peak_2)

        if (len(peak2_dimension[0])>1):
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][1]
        else:
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][0]

        val={'tempo':i,f'<>({V})':mean,f'var({V})':var,f'kurt({V})':kurt,f'RMS({V})':rms,f'freq1({V})':xf[index1],f'freq2({V})':xf[index2],f'amp1({V})':h_max_peak_1,f'amp2({V})':h_max_peak_2}
        tab=tab.append(val,ignore_index=True)
    return tab

In [22]:
from functools import reduce
GyrX=time_freq_feat('GyrX')
GyrY=time_freq_feat('GyrY')
GyrZ=time_freq_feat('GyrZ')
AccX=time_freq_feat('AccX')	
AccY=time_freq_feat('AccY')
AccZ=time_freq_feat('AccZ')
Roll=time_freq_feat('Roll')
Pitch=time_freq_feat('Pitch')
Yaw=time_freq_feat('Yaw')
rpm_ESC_0=time_freq_feat('RPM(ESC_0)')
curr_ESC_0=time_freq_feat('Curr(ESC_0)')
rpm_ESC_1=time_freq_feat('RPM(ESC_1)')
curr_ESC_1=time_freq_feat('Curr(ESC_1)')
rpm_ESC_2=time_freq_feat('RPM(ESC_2)')
curr_ESC_2=time_freq_feat('Curr(ESC_2)')
rpm_ESC_3=time_freq_feat('RPM(ESC_3)')
curr_ESC_3=time_freq_feat('Curr(ESC_3)')
rpm_ESC_4=time_freq_feat('RPM(ESC_4)')
curr_ESC_4=time_freq_feat('Curr(ESC_4)')
rpm_ESC_5=time_freq_feat('RPM(ESC_5)')
curr_ESC_5=time_freq_feat('Curr(ESC_5)')
merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,rpm_ESC_0,curr_ESC_0,rpm_ESC_1,curr_ESC_1,rpm_ESC_2,curr_ESC_2,rpm_ESC_3,curr_ESC_3,rpm_ESC_4,curr_ESC_4,rpm_ESC_5,curr_ESC_5]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['tempo'],how='outer'), merged)
df_merged["Guasto"] = guasto
display(df_merged)

d:\Program Files\Python311\Lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\mgale\AppData\Local\Temp\ipykernel_2120\216894438.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
d:\Program Files\Python311\Lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\mgale\AppData\Local\Temp\ipykernel_2120\216894438.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
d:\Program Files\Python311\Lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Ca

,tempo,<>(GyrX),var(GyrX),kurt(GyrX),RMS(GyrX),freq1(GyrX),freq2(GyrX),amp1(GyrX),amp2(GyrX),<>(GyrY),...,amp2(RPM(ESC_5)),<>(Curr(ESC_5)),var(Curr(ESC_5)),kurt(Curr(ESC_5)),RMS(Curr(ESC_5)),freq1(Curr(ESC_5)),freq2(Curr(ESC_5)),amp1(Curr(ESC_5)),amp2(Curr(ESC_5)),Guasto
0,0.0,-0.141540,0.034814,-0.873153,0.233984,104.0,98.0,11.235387,6.653416,0.014573,...,7021.681589,3.342764,0.497704,0.079588,3.416190,86.0,89.0,4.992379,4.487772,0
1,1.0,-0.141375,0.034725,-0.867272,0.233694,99.0,93.0,11.157941,6.521929,0.014483,...,7117.055103,3.345114,0.498230,0.085199,3.418567,94.0,97.0,4.706958,4.149592,0
2,2.0,-0.141245,0.034632,-0.861153,0.233418,108.0,103.0,11.054809,6.191746,0.014298,...,7102.485383,3.347450,0.498742,0.090808,3.420929,74.0,77.0,4.328675,3.797077,0
3,3.0,-0.141366,0.034539,-0.856372,0.233293,97.0,92.0,10.849367,5.609436,0.014201,...,7078.252888,3.349774,0.499241,0.096416,3.423276,85.0,84.0,3.952165,3.862416,0
4,4.0,-0.141284,0.034444,-0.850090,0.233041,100.0,95.0,10.702681,4.966362,0.014196,...,7044.290678,3.352085,0.499726,0.102023,3.425608,87.0,86.0,4.696883,3.314994,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,76.0,-0.116353,0.033516,-0.653034,0.216738,0.0,137.0,34.578421,9.522904,-0.007438,...,5002.202010,3.489110,0.508393,0.539209,3.561052,105.0,106.0,10.032996,2.975712,0
77,77.0,-0.115605,0.033677,-0.652457,0.216710,0.0,134.0,35.099089,9.767665,-0.007751,...,4886.359658,3.489813,0.507414,0.549690,3.561604,101.0,102.0,9.310800,2.673989,0
78,78.0,-0.114927,0.033796,-0.653701,0.216622,0.0,141.0,35.587071,9.973493,-0.007897,...,4769.995034,3.490513,0.506438,0.560176,3.562153,105.0,106.0,8.582048,2.385153,0
79,79.0,-0.114616,0.033758,-0.650553,0.216372,0.0,146.0,35.914340,10.012381,-0.007750,...,4654.350073,3.491209,0.505466,0.570668,3.562700,101.0,102.0,7.855755,2.112371,0


In [23]:
path_file = path_file.replace(r"M2\NO_FAULT2", "")
os.chdir(path_file)
df_merged.to_csv('m2_nofault.csv', index=False)
print(path_file)

d:\GitHub\ProgettoManutenzionePreventiva\
